<h3> Formatting the output from mi-faser so that it can be used in weka </h3>

First, we will read in each of the mi-faser output files and store them in a list of dictionaries

In [28]:
from collections import defaultdict

samples_list = ['N2', 'N3', 'N4','E1','E3','E4','E5']

counts_list = []

for sample in samples_list:
    sample_dict = defaultdict(int)
    with open("mifaser/" + sample + '_mifaser.tsv') as fn:
        lines = fn.readlines()
        for line in lines:
            sample_dict[line.split('\t')[0]] = int(line.split('\t')[1])
    
    counts_list.append(sample_dict)     

Next, we will create a list of all of the unitque enzyme accession numbers

In [29]:
all_ec = []

for counts in counts_list:
    for ec in counts.keys():
        if ec not in all_ec:
            all_ec.append(ec)

In [35]:
counts = []
for t in range(len(all_ec)):
    counts.append([0,0,0,0,0,0,0])

In [51]:
for z in range(len(counts_list)):
    total = sum(counts_list[z].values())
    for ecNumber in counts_list[z].keys():
        index = all_ec.index(ecNumber)
        norm = (counts_list[z][ecNumber] / total) * 100000
        counts[index][z] = float("%.3f" % norm)

In [43]:
print(counts[0])
print(counts[1])

['0.174', 0, '1.849', '2.675', '23.822', 0, 0]
['25.553', 0, '280.318', '0.892', 0, 0, 0]


In [47]:
experimental_group = ['Normal','Normal','Normal','Experimental','Experimental','Experimental','Experimental']

with open('mifaser_formatted_output.csv','w') as fn:
    fn.write('Sample,Experimental_Group,')
    for EC in all_ec[:-1]:
        fn.write(str(EC) + ',')
    fn.write(str(all_ec[-1]))
    fn.write('\n')
    for g in range(len(counts[0])):
        fn.write(samples_list[g]+',')
        fn.write(experimental_group[g]+',')
        for n in range(len(counts)):
            if n != len(counts)-1:
                fn.write(str(counts[n][g])+',')
            else:
                fn.write(str(counts[n][g]))
        fn.write('\n')

Get the most descriptive enzymes from weka

In [48]:
with open('weka_output.txt') as fn:
    ecs = [line.strip('\n').split(' ')[-1] for line in fn.readlines()]

In [65]:
with open("top_ten.csv","w") as output:
    output.write("Normal,High\n")
    for enzC in ecs[:10]:
        ecIndex = all_ec.index(enzC)
        avgNormal = sum(counts[ecIndex][:3]) / 3
        avgExp = sum(counts[ecIndex][3:]) / 4

        output.write('{:.3f},{:.3f}\n'.format(avgNormal, avgExp))

In [64]:
testList = [0,1,2,3]
print(testList[:3])

[0, 1, 2]


In [75]:
with open("mifaser_heatmap_data.csv","w") as fN:
    fN.write(",N2,N3,N4,E1,E3,E4,E5\n")
    for enzC in ecs[:10]:
        ecIndex = all_ec.index(enzC)
        fN.write(all_ec[ecIndex] + ',')
        for y in range(len(counts[ecIndex])):
            if y != len(counts[ecIndex]) -1:
                fN.write(str(counts[ecIndex][y]) + ',')
            else:
                fN.write(str(counts[ecIndex][y]))
        fN.write('\n')